In [1]:
#user_data
%%writefile series.csv
user_id,series,season,timestamp,genre,duration_mins
521,”Mirzapur”,3,2024-07-30 15:00:00,action,300
672,”Panchayat”,3,2024-07-30 15:00:00,comedy,200
197,”Family Man”,2,2024-07-30 15:00:00,action,500
521,”Mirzapur”,2,2024-07-29 15:00:00,action,280
211,”Queens Gambit”,1,2024-07-30 15:00:00,drama,170
521,”Mirzapur”,1,2024-07-28 15:00:00,action,230
844,”Westworld”,3,2024-07-30 15:00:00,sci-fi,310
672,”Panchayat”,3,2024-07-29 15:00:00,comedy,210
256,”Homecoming”,2,2024-07-30 15:00:00,thriller,310
489,”Outer Range”,1,2024-07-30 15:00:00,sci-fi,340
200,”Black Mirror”,2,2024-07-30 15:00:00,sci-fi,140
256,”Outer Range”,2,2024-07-30 15:00:00,thriller,250
489,”Outer Range”,2,2024-07-28 15:00:00,sci-fi,170
200,”Black Mirror”,3,2024-07-29 15:00:00,sci-fi,190
672,”Panchayat”,2,2024-07-28 15:00:00,comedy,160
672,”Outer Range”,1,2024-07-25 15:00:00,sci-fi,250
200,”Black Mirror”,4,2024-07-28 15:00:00,sci-fi,200
844,”Westworld”,2,2024-07-29 15:00:00,sci-fi,300
672,”Black Mirror”,5,2024-07-28 15:00:00,sci-fi,150
672,”Panchayat”,1,2024-07-27 15:00:00,comedy,190

Writing series.csv


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=faabc7f4197ca7641e379ea481ac4ad6a474ec6dc282cf427c13ed263fc76f32
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
#Import Libraries
from pyspark.sql import SparkSession

In [4]:
#Create SparkSession for app (Streaming Analysis)
lti_spark=SparkSession.builder.appName('akshitha').getOrCreate()

In [5]:
#Create Dataframe series_df
series_df=lti_spark.read.option('Header','True').csv('series.csv')

In [6]:
series_df.show()

+-------+---------------+------+-------------------+--------+-------------+
|user_id|         series|season|          timestamp|   genre|duration_mins|
+-------+---------------+------+-------------------+--------+-------------+
|    521|     ”Mirzapur”|     3|2024-07-30 15:00:00|  action|          300|
|    672|    ”Panchayat”|     3|2024-07-30 15:00:00|  comedy|          200|
|    197|   ”Family Man”|     2|2024-07-30 15:00:00|  action|          500|
|    521|     ”Mirzapur”|     2|2024-07-29 15:00:00|  action|          280|
|    211|”Queens Gambit”|     1|2024-07-30 15:00:00|   drama|          170|
|    521|     ”Mirzapur”|     1|2024-07-28 15:00:00|  action|          230|
|    844|    ”Westworld”|     3|2024-07-30 15:00:00|  sci-fi|          310|
|    672|    ”Panchayat”|     3|2024-07-29 15:00:00|  comedy|          210|
|    256|   ”Homecoming”|     2|2024-07-30 15:00:00|thriller|          310|
|    489|  ”Outer Range”|     1|2024-07-30 15:00:00|  sci-fi|          340|
|    200| ”B

In [14]:
#Find the user with maximum watchtime
series_df.groupBy('user_id').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).limit(1).collect()[0][0]

'672'

In [10]:
#Calculate overall total Watchtime
series_df.agg({'duration_mins':'sum'}).collect()[0][0]

4850.0

In [17]:
#Find most popular shows (based on watchtime)
df=series_df.groupBy('series').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).limit(3)
df.rdd.flatMap(lambda x:[x[0]]).collect()

['”Outer Range”', '”Mirzapur”', '”Panchayat”']

In [21]:
#Find most popular shows (based on user popularity)
df=series_df.groupBy('series').agg({'user_id':'count'}).orderBy('count(user_id)',ascending=False).limit(3)
df.rdd.flatMap(lambda x:[x[0]]).collect()

['”Outer Range”', '”Panchayat”', '”Black Mirror”']

In [35]:
#Find the most popular genre
series_df.groupBy('genre').agg({'user_id':'count'}).orderBy('count(user_id)',ascending=False).limit(1).collect()[0][0]

'sci-fi'

In [28]:
#Calculate total watchtime per user
series_df.groupBy('user_id').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)').withColumnRenamed('sum(duration_mins)','watchtime').show()

+-------+---------+
|user_id|watchtime|
+-------+---------+
|    211|    170.0|
|    197|    500.0|
|    489|    510.0|
|    200|    530.0|
|    256|    560.0|
|    844|    610.0|
|    521|    810.0|
|    672|   1160.0|
+-------+---------+



In [38]:
#Find most popular genre (based on engagement count)
series_df.groupBy('genre').agg({'duration_mins':'sum'}).orderBy('sum(duration_mins)',ascending=False).limit(1).collect()[0][0]

'sci-fi'

In [55]:
#Find average watchtime per genre
import pyspark.sql.functions as f
df=series_df.groupBy('genre').agg({'duration_mins':'avg'}).orderBy('avg(duration_mins)')
df.withColumn('avg(duration_mins)',f.round('avg(duration_mins)',1)).show()

+--------+------------------+
|   genre|avg(duration_mins)|
+--------+------------------+
|   drama|             170.0|
|  comedy|             190.0|
|  sci-fi|             227.8|
|thriller|             280.0|
|  action|             327.5|
+--------+------------------+



In [73]:
#Find peak traffic days
#(Output 1 = Full Date)
df=series_df.groupBy('timestamp').agg({'user_id':'count'}).orderBy('count(user_id)',ascending=False).limit(1)

#(Output 2 = Only Day)
df.withColumn('timestamp',f.dayofmonth('timestamp')).collect()[0][0]

30

In [76]:
#Find the user with most diverse show preference
series_df.groupBy('user_id').agg({'series':'count'}).orderBy('count(series)',ascending=False).limit(1).collect()[0][0]

'672'

In [109]:
#Find the binge-watchers
df1=series_df.groupBy('user_id','timestamp','series').agg({'season':'count'}).withColumnRenamed('count(season)','count').orderBy('user_id')
df2=series_df.groupBy('series').agg({'season':'max'}).withColumnRenamed('max(season)','max')
df1.show()
df2.show()
df3=df1.join(df2,'series','inner')
df3.show()
df3.filter(f.col('count')==f.col('max')).collect()[0][1]

+-------+-------------------+---------------+-----+
|user_id|          timestamp|         series|count|
+-------+-------------------+---------------+-----+
|    197|2024-07-30 15:00:00|   ”Family Man”|    1|
|    200|2024-07-29 15:00:00| ”Black Mirror”|    1|
|    200|2024-07-28 15:00:00| ”Black Mirror”|    1|
|    200|2024-07-30 15:00:00| ”Black Mirror”|    1|
|    211|2024-07-30 15:00:00|”Queens Gambit”|    1|
|    256|2024-07-30 15:00:00|   ”Homecoming”|    1|
|    256|2024-07-30 15:00:00|  ”Outer Range”|    1|
|    489|2024-07-28 15:00:00|  ”Outer Range”|    1|
|    489|2024-07-30 15:00:00|  ”Outer Range”|    1|
|    521|2024-07-30 15:00:00|     ”Mirzapur”|    1|
|    521|2024-07-28 15:00:00|     ”Mirzapur”|    1|
|    521|2024-07-29 15:00:00|     ”Mirzapur”|    1|
|    672|2024-07-28 15:00:00| ”Black Mirror”|    1|
|    672|2024-07-25 15:00:00|  ”Outer Range”|    1|
|    672|2024-07-27 15:00:00|    ”Panchayat”|    1|
|    672|2024-07-29 15:00:00|    ”Panchayat”|    1|
|    672|202

'211'

In [125]:
#Find the user with longest watching streak
series_df.groupBy('user_id').agg({'timestamp':'count'}).orderBy('count(timestamp)',ascending=False).limit(1).collect()[0][0]

'672'

In [112]:
#Total Seasons available
df2=series_df.groupBy('series').agg({'season':'max'}).withColumnRenamed('max(season)','max')
df2.agg({'max':'sum'}).collect()[0][0]

21.0

In [116]:
#Fetch a list of all series
df=series_df.groupBy('series').agg({'season':'max'})
df.rdd.flatMap(lambda x:[x[0]]).collect()

['”Black Mirror”',
 '”Family Man”',
 '”Homecoming”',
 '”Mirzapur”',
 '”Outer Range”',
 '”Panchayat”',
 '”Queens Gambit”',
 '”Westworld”']